# Creating a Siamese Network

Creating a Siamese Network Model in Keras with a custom layer and a custom loss function involves several steps. We'll start by defining the custom layer and the custom loss function, and then proceed to build the Siamese Network.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Layer, Input, Dense, Flatten, Lambda
from tensorflow.keras.models import Model

class CustomLayer(Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1],self.units),
                                 initializer='random_normal',
                                 trainable=True)# Create a simple trainable weight initialized randomly

    def call(self, inputs):
        return tf.matmul(input,self.weights)  # Apply to input


In [13]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense, Flatten, Lambda
from tensorflow.keras.models import Model

# Contrastive loss function
def contrastive_loss(y_true, y_pred):
    margin = 1.0
    y_true = tf.cast(y_true, tf.float32)  # Cast labels to float
    square_pred = tf.square(y_pred)  # (D)^2 for similar pairs
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))  # (max(margin - D, 0))^2 for dissimilar pairs
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)


A Siamese Network consists of two identical subnetworks with shared weights.

In [14]:
def build_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    return Model(input, x)


input_shape = (28, 28)
base_network = build_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Euclidean distance layer
distance = Lambda(lambda embeddings: tf.sqrt(
    tf.reduce_sum(tf.square(embeddings[0] - embeddings[1]), axis=1, keepdims=True)
))([processed_a, processed_b])

# Siamese model
model = Model([input_a, input_b], distance)

# Compile with custom loss
model.compile(loss=contrastive_loss, optimizer='adam')


Let's create some dummy data to train the Siamese Network. In a real-world scenario, this would be pairs of images and a label indicating whether they are similar or not. For simplicity, we will create random numpy arrays as dummy data. The concept remains the same for actual image data.

In [15]:
def generate_dummy_data(num_samples=1000, input_shape=(28, 28)):
    data = np.random.random((num_samples, *input_shape))
    labels = np.random.randint(0, 2, (num_samples, 1))
    return data, labels

num_samples = 1000
data_a, labels_a = generate_dummy_data(num_samples, input_shape)
data_b, labels_b = generate_dummy_data(num_samples, input_shape)

# Label = 1 if same, else 0
labels = (labels_a == labels_b).astype(float)


In [16]:
model.fit([data_a, data_b], labels, epochs=10, batch_size=128)


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 1.7858
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3039 
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2487 
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2185 
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1926 
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1706 
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1576 
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1382 
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1247 
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1105 
